In [1]:
import csv
import pandas as pd
import numpy as np
import re
import datetime as dt
import time

In [2]:
df = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv", parse_dates=[0], infer_datetime_format=True)

In [3]:
df['fullDate'] = df['Event Clearance Date'].apply(lambda x: str(x).split(' ')[0])
df['ECG'] = df['Event Clearance Group']
df['dtDate'] = pd.DatetimeIndex(df['Event Clearance Date'])
df['eventClearanceSeconds'] = df['dtDate'].apply(lambda x: x.hour * 3600 + x.minute*60)

In [4]:
df['weekday'] = df['dtDate'].apply(lambda x: x.weekday())
df['weekdayName'] = df['dtDate'].apply(lambda x: x.weekday_name)

weekDummies = pd.get_dummies(df['weekdayName'])
df = pd.concat([df,weekDummies], axis = 1)

zoneBeatDummies = pd.get_dummies(df['Zone/Beat'])
df = pd.concat([df,zoneBeatDummies], axis = 1)

In [5]:
import calendar
weather = pd.read_csv("WEATHER.csv")

months_map = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr':'04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct':'10', 'Nov': '11', 'Dec': '12'}
weather['m'] = weather['month'].apply(lambda x: months_map[x])
weather['day'] = weather['date'].apply(lambda x: ('0' + str(x)) if x < 10 else str(x) )
weather['fullDate'] = weather.apply(lambda x: str(x['m']) + '/' + str(x['day']) + '/' + str(x['\ufeffyear']), axis=1 )
weather['clear'] = weather['events'].apply(lambda x: 1 if ('Rain' in str(x) or 'Snow' in str(x) or 'Fog' in str(x) or 'Thunderstorm' in str(x)) else 0)
weather['notClear'] = weather['clear'].apply(lambda x: 1 if x is 0 else 0)

In [6]:
merged = pd.merge(df, weather, on = 'fullDate', how = 'left')

In [7]:
#list(merged.columns.values)

In [8]:
merged.shape

(1335033, 151)

In [9]:
merged = merged.dropna(subset=["Event Clearance Date", "Event Clearance Group","date", "tempAvg"])
merged.shape

(1311452, 151)

In [10]:
#merged.groupby('ECG').count()

In [102]:
sample = []
foo = merged.groupby('ECG')
for name, group in foo:
    sample.append(group.sample(n=min(10000, group.shape[0]), replace = False, random_state = 14, ))
new_df = pd.concat(sample)
new_df['secondsInDay'] = new_df['eventClearanceSeconds'].apply(lambda x: int(x))

In [103]:
#new_df.groupby('ECG').count()
new_df.shape

(311269, 152)

In [104]:
train=new_df.sample(frac=0.7,random_state=14)
test=new_df.sample(frac = 0.3, random_state = 16)

In [105]:
train.shape

(217888, 152)

In [106]:
test.shape

(93381, 152)

In [107]:
features = ['secondsInDay', 'clear', 'Latitude', 'Longitude', 'm', 'day', 'tempHigh',
 'tempAvg',
 'tempLow',
 'dewHigh',
 'dewAvg',
 'dewLow',
 'humidityHigh',
 'humidityAvg',
 'humidityLow',
 'seaLvlHigh',
 'seaLvlAvg',
 'seaLvlLow',
 'visibilityHigh',
 'visibilityAvg',
 'visibilityLow',
 'windHigh',
 'windAvg','Friday',
 'Monday',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday','99',
 'B1',
 'B2',
 'B3',
 'BS',
 'C1',
 'C2',
 'C3',
 'CCD',
 'COMM',
 'CS',
 'CTY',
 'D1',
 'D2',
 'D3',
 'DET',
 'DS',
 'E',
 'E1',
 'E2',
 'E3',
 'EDD',
 'EP',
 'ES',
 'F1',
 'F2',
 'F3',
 'FS',
 'G1',
 'G2',
 'G3',
 'GS',
 'H2',
 'H3',
 'INV',
 'J1',
 'J2',
 'J3',
 'JS',
 'K1',
 'K2',
 'K3',
 'KCIO07',
 'KS',
 'L1',
 'L2',
 'L3',
 'LAPT',
 'LS',
 'M1',
 'M2',
 'M3',
 'MS',
 'N',
 'N1',
 'N2',
 'N3',
 'NP',
 'NS',
 'O1',
 'O2',
 'O3',
 'OS',
 'Q1',
 'Q2',
 'Q3',
 'QS',
 'R1',
 'R2',
 'R3',
 'RS',
 'S',
 'S1',
 'S2',
 'S3',
 'SCTR1',
 'SP',
 'SS',
 'TAC3',
 'TRF',
 'U1',
 'U2',
 'U3',
 'US',
 'W',
 'W1',
 'W2',
 'W3',
 'WP',
 'WS']
X = train[features]
X2 = test[features]
X.head()

,secondsInDay,clear,Latitude,Longitude,m,day,tempHigh,tempAvg,tempLow,dewHigh,...,U1,U2,U3,US,W,W1,W2,W3,WP,WS
399073,27000,0.0,47.614421,-122.346308,03,25,56.0,46.0,36.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081801,62400,0.0,47.596657,-122.327680,08,25,91.0,76.0,61.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
856128,61500,0.0,47.723795,-122.292841,09,28,69.0,61.0,53.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194062,85080,0.0,47.601719,-122.332874,09,16,64.0,58.0,52.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
776252,51480,1.0,47.596657,-122.327680,01,27,58.0,54.0,49.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
y = train.ECG
y2 = test.ECG
print(X.shape)
print(y.shape)

(217888, 120)
(217888,)


In [109]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(solver='lbfgs', multi_class = 'multinomial', max_iter=200, class_weight= 'balanced')
fit = lr.fit(X, y)

In [110]:
preds = lr.predict(X2)

In [111]:
test['ECG_pred'] = preds
fc = ['ECG', 'ECG_pred']
acc = test[fc]

In [112]:
acc['int_accuracy'] = (acc['ECG'] == acc['ECG_pred']).astype(int)

C:\Users\Tin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [113]:
acc.sum(axis=0)

ECG             BEHAVIORAL HEALTHACCIDENT INVESTIGATIONTRESPAS...
ECG_pred        VICE CALLSVICE CALLSPUBLIC GATHERINGSHARBOR CA...
int_accuracy                                                 1684
dtype: object

In [114]:
1684/93381

0.018033647101658796

In [115]:
acc.groupby('ECG_pred').count()

,ECG,int_accuracy
ECG_pred,,
ACCIDENT INVESTIGATION,11580,11580
ANIMAL COMPLAINTS,109,109
ARREST,134,134
ASSAULTS,1,1
AUTO THEFTS,10,10
BEHAVIORAL HEALTH,4751,4751
BIKE,880,880
CAR PROWL,15,15
DISTURBANCES,1793,1793


In [116]:
acc.shape

(93381, 3)

In [117]:
303/11638

0.02603540127169617